In [1]:
from config import AU_ORDER, JAANET_path, PREPROCESSED_AU_TEMPLATE, VIDEO_NUMBERS, PREPROCESSED_META, FRAME_TEMPLATE, PREPROCESSED_VIDEO_TEMPLATE, fold_assignment, JAANET_FOLDS
import tensorflow as tf
import pandas as pd
import os

In [5]:
JAANET_path = os.path.join('..', JAANET_path)
os.path.exists(JAANET_path)

True

In [2]:
def labels_to_csv():
    # fold_assignment_dict = fold_assignment(assignment_type=assignement_type)
    columns = ['frame'] + ['AU{}'.format(au) for au in AU_ORDER]
    dfs = []
    for video_number in VIDEO_NUMBERS:
        print(video_number)
        video_df = pd.read_csv(header=None,
                               names=columns,
                               filepath_or_buffer=PREPROCESSED_AU_TEMPLATE.format(video_number)
                              )
        video_df['video'] = video_number
        dfs.append(video_df)
     
    df = pd.concat(dfs)
    # df.to_csv(path_or_buf=PREPROCESSED_META.format(assignement_type),
              # index=False,
              # columns=columns + ['video', 'fold'])
    return df

def jaanet_fold_csv():
    full_csv = labels_to_csv()
    folds_csv = []
    
    for i in range(len(JAANET_FOLDS)):
        file_csv = pd.read_csv(filepath_or_buffer=JAANET_FOLDS[i], sep='/', header=None, names=['ds', 'video', 'frame'])
        file_csv['video'] = file_csv['video'].map(lambda x: x[2:])
        file_csv['frame'] = file_csv['frame'].map(lambda x: int(x[:-4]) + 1)
        file_csv = file_csv.drop(['ds'], axis=1)
        fold_csv = pd.merge(full_csv, file_csv, on=['frame', 'video'])
        fold_csv['path'] = fold_csv.apply(lambda x: os.path.join(PREPROCESSED_VIDEO_TEMPLATE + x['video'],
                                                                 FRAME_TEMPLATE + str(x['frame']) + '.jpeg'), axis=1)
        
        fold_csv = fold_csv.drop(['video', 'frame'], axis=1)
        folds_csv.append(fold_csv)
        
        fold_csv.to_csv(path_or_buf=os.path.join(JAANET_path, 'fold{}.csv'.format(i)),
                        index=False,
                        header=True)
        
    fold_01 = pd.concat([folds_csv[0], folds_csv[1]], ignore_index=True)
    fold_01.to_csv(path_or_buf=os.path.join(JAANET_path, 'fold01.csv'),
                   index=False,
                   header=True)
    fold_02 = pd.concat([folds_csv[0], folds_csv[2]], ignore_index=True)
    fold_02.to_csv(path_or_buf=os.path.join(JAANET_path, 'fold02.csv'),
                   index=False,
                   header=True)
    fold_12 = pd.concat([folds_csv[1], folds_csv[2]], ignore_index=True)
    fold_12.to_csv(path_or_buf=os.path.join(JAANET_path, 'fold12.csv'),
                   index=False,
                   header=True)
        
    
    return fold_csv



            
# jaanet_fold_csv()['path'][0]


In [8]:
L = [1, 2, 3]
L[[0, 2]]

TypeError: list indices must be integers or slices, not list

In [24]:
labels_to_csv()

001
002
003
004
005
006
007
008
009
010
011
012
013
016
017
018
021
023
024
025
026
027
028
029
030
031
032


,frame,AU12,AU17,AU20,AU26,AU4,AU6,AU15,AU1,AU25,AU2,AU5,AU9,video
0,1,2,0,0,0,0,0,0,0,0,0,0,0,001
1,2,2,0,0,0,0,0,0,0,0,0,0,0,001
2,3,2,0,0,0,0,0,0,0,0,0,0,0,001
3,4,2,0,0,0,0,0,0,0,0,0,0,0,001
4,5,2,0,0,0,0,0,0,0,0,0,0,0,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4840,4841,0,0,0,0,5,3,0,0,3,0,0,5,032
4841,4842,0,0,0,0,5,3,0,0,3,0,0,5,032
4842,4843,0,0,0,0,5,3,0,0,3,0,0,5,032
4843,4844,0,0,0,0,5,3,0,0,3,0,0,5,032


In [8]:
def labels_to_csv(assignement_type='classic'):
    fold_assignment_dict = fold_assignment(assignment_type=assignement_type)
    columns = ['frame'] + ['AU{}'.format(au) for au in AU_ORDER]
    dfs = []
    for video_number in VIDEO_NUMBERS:
        print(video_number)
        video_df = pd.read_csv(header=None,
                               names=columns,
                               filepath_or_buffer=PREPROCESSED_AU_TEMPLATE.format(video_number)
                              )
        video_df['video'] = video_number
        video_df['fold'] = fold_assignment_dict[video_number]
        dfs.append(video_df)
     
    df = pd.concat(dfs)
    df.to_csv(path_or_buf=PREPROCESSED_META.format(assignement_type),
              index=False,
              columns=columns + ['video', 'fold'])
    
    

    
def assign_fold(assignment_type):
    fold_assignment_dict = fold_assignment(assignment_type=assignment_type)
    def fun(video, labels):
        return video, labels[0], labels[1:], fold_assignment_dict[video]
    return fun

def filter_fold(folds):
    folds = tf.constant(folds)
    n_folds = tf.shape(folds)[0]
    def fun(*args):
        fold = args[-1]
        return tf.math.reduce_prod(fold - folds) == 0
    return fun

def separating_map(*args):
    return args[0], args[1:]

def decode_img(img):
    img = tf.image.decode_jpeg(tf.io.read_file(img), channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def load_image(*args):
    labels = args[:-1]
    path = args[-1]
    tf.print(path)

    return decode_img(path), labels


    
    

def load_dataset_meta(folds, assignment_type='jaanet'):
    folds = sorted(folds)
    folds = [str(fold) for fold in folds]
    if assignment_type == 'jaanet':
        str_fold = ''.join(folds)
        dataset_file = os.path.join(JAANET_path, 'fold{}.csv'.format(str_fold))
        dataset_types = 12 * [float()] + [str()]
        return (tf.data.experimental.CsvDataset(dataset_file,
                                                record_defaults=dataset_types,
                                                header=True))
        
    dataset_file = PREPROCESSED_META.format(assignment_type)
    dataset_types = [str()] + 12 * [float()] + [str(), int()]
    return (tf.data.experimental.CsvDataset(dataset_file,
                                            record_defaults=dataset_types,
                                            header=True)
            .filter(filter_fold(folds)))

def gen_disfa(folds, n_au, batch_size, mode, assignement_type='jaanet'):
    pass

def gen_disfa01
    

# disfa = load_dataset_meta([2]).map(load_image).shuffle(1000)


# for el in disfa:
    # pass
    #tf.print(el)


,AU12,AU17,AU20,AU26,AU4,AU6,AU15,AU1,AU25,AU2,AU5,AU9,path
0,2,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN001/f...
1,2,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN001/f...
2,2,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN001/f...
3,2,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN001/f...
4,2,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN001/f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78652,0,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN031/f...
78653,0,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN031/f...
78654,0,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN031/f...
78655,0,0,0,0,0,0,0,0,0,0,0,0,../resources/DISFA/preprocessed/images/SN031/f...


In [23]:
def hypertuning_fold_ds(folds):
    fold_file = os.path.join(JAANET_path, 'fold{}.csv'.format(folds))
    columns = ['AU{}'.format(au) for au in AU_ORDER] + ['path']
    video_df = pd.read_csv(header=0,
                           names=columns,
                           filepath_or_buffer=fold_file)
    return video_df

def extract_video(path):
    return path.split('/')[-2]

fold01 = hypertuning_fold_ds('01')
fold01['video'] = fold01['path'].map(extract_video)
fold01.groupby('video').sum()

videos = ["SN001", "SN002", "SN006", "SN009", "SN010", "SN011", "SN012", "SN013", "SN016", "SN018", "SN021", "SN024", "SN026", "SN027", "SN028", "SN030", "SN031", "SN032"]

fold01_train = fold01[fold01['video'].isin(videos[:12])].drop('video', axis=1)

fold01_train.to_csv(os.path.join(JAANET_path, 'hypertuning', 'fold01_train.csv'),
                    index=False,
                    header=True)
fold01_valid.to_csv(os.path.join(JAANET_path, 'hypertuning', 'fold01_valid.csv'),
                    index=False,
                    header=True)

